# Chapter 20. Concurrent Execution

This chapter focuses on `concurrent.futures.Executor` classes that encapuslate the pattern of "spawning a bunch of independent threads and collecting the results in a queue"

Here the author introduces the concept of "futures"--objects representing the asynchronous execution of an operation, similar to JS promises.

Executors are the most important high-level feature while futures are low-level objects.

## Concurrent Web Downloads

Concurrency is essential for efficient network I/O: instead of idly waiting for remote machines, the application should do sth else until a response comes back.

Three simples programs to download images of 20 country flags from the web
 - `flags.py`: runs sequentially
 - `flags_threadpool.py` uses the `concurrent.futures` pacakge
 - `flags_asyncio.py` uses the `asyncio`

### A Sequential Download Script

In [3]:
!pip install httpx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.1 MB/s eta 0:00:00


In [ ]:
import time
from pathlib import Path
from typing import Callable

import httpx

POP20_CC = ('CN IN US ID BR PK NG BD RU KR '
            'MX PH VN ET EG DE IR TR CD FR').split()

BASE_URL = 'https://www.fluentpython.com/data/flags'
DEST_DIR = Path('downloaded')

# save the img bytes to filename in the DEST_DIR
def save_flag(img: bytes, filenames: str) -> None:
  (DEST_DIR / filename).write_bytes(img)

def get_flag(cc: str) -> bytes:
  url = f'{BASE_URL}/{cc}/{cc}.gif'.lower()
  resp = httpx.get(url, timeout=6.1, # it's a good practice to add a sensible timeout
                   follow_redirects=True) # By default, HTTPX does not follow redirects
  resp.raise_for_status()
  return resp.content

# key function to compare with the concurrent implementations
def download_many(cc_list: list[str]) -> int:
  for cc in sorted(cc_list):
    image = get_flag(cc)
    save_flag(image, f'{cc}.gif')
    print(cc, end=" ", flush=True) # display one country code at a time in the same line
                                   # flush=True is needed because by default Python output is line buffered
  return len(cc_list)

def main(downloader: Callable[[list[str]], int]) -> None:
  DEST_DIR.mkdir(exist_ok=True)
  t0 = time.perf_counter()
  count = downloader(POP20_CC)
  elapsed = time.perf_counter() - t0
  print(f"\n{count} downloads in {elapsed:.2f}s")

if __name__ == "__main__":
  main(download_many)


In [ ]:
!python flags.py

BD BR CD CN DE EG ET FR ID IN IR KR MX NG PH PK RU TR US VN 
20 downloads in 6.43s


### Downloading with concurrent.futures

Main Features: `ThreadPoolExecutor` and `ProcessPoolExectuor` classes, which implement an API to submit cllables for execution in different threads or processes. The classes transparently manage a pool of worker threads or processes, and queues to distribute jobs and collect results.

In [ ]:
# flags_threadpool.py
from concurrent import futures

from flags import save_flag, get_flag, main

def download_one(cc: str):
  image = get_flag(cc)
  save_flag(image, f'{cc}.gif')
  print(cc, end=" ", flush=True)
  return cc

def download_many(cc_list: list[str]) -> int:
  # instantiate the thread pool executor as a context manager
  # exectuor.__exit__ will call executor.shutdown(wait=True) which will block until all threads are done
  with futures.ThreadPoolExecutor() as executor:
    res = executor.map(download_one, sorted(cc_list))

  return len(list(res))

if __name__ == "__main__":
  main(download_many)



In [ ]:
!python flags_threadpool.py

BD CN DE BR EG CD ET ID FR KR IN IR MX NG PH RU PK TR US VN 
20 downloads in 1.29s


In [ ]:
import os
os.cpu_count() + 4

6

### Where are teh Futures?

Futures are core components of `concurrent.futures` and of `asyncio` but as users of these libraries we sometimes don't see them.

There are two classes naemd `Future` in the standard library: `concurrent.futures.Future` and `asyncio.Future`. They serve the same purpose: an instance of either `Future` class represents a deferred computation that may or may not have completed.

Futures encapsulate pending operations so that we can put them in queues, check whether they are done, and retrieve results when they become available.

We should not create them: they are meant to be instantiated exclusively by the concurrency framework, be it `concurrent.futures` or `asyncio`. Why? a `Future` represents something that will eventually run, therefore it must be scheduled to run, and that's the job of the framework.

Application code is not supposed to change the state of the a future: the concurrency framework changes the state of a future when the computation it represents is done, and we can't control when that happens.

Both types of `Future` have a `.done()` method that is nonblocking and returns a Boolean that tells you whether the callable wrapped by that future has executed or not. However, instaed of repeatedly asking whether a future is done, client code usually asks to be notified. That's why both `Future` classes have an `.add_done_callback()` method.

There is also a `.result()` method, which works the same in both classes when the future is done: it returns the result of the callable, or re-raises whatever exception might have been thrown when the callable was executed. However, when the future is not done, the behavior of the `result` method is very different.

In [ ]:
# flags_threadpool_futures.py
from concurrent import futures

from flags import save_flag, get_flag, main

def download_one(cc: str):
  image = get_flag(cc)
  save_flag(image, f'{cc}.gif')
  print(cc, end=" ", flush=True)
  return cc

def download_many(cc_list: list[str]) -> int:

  cc_list = cc_list[:5] # just for this demonstration
  # instantiate the thread pool executor as a context manager
  # exectuor.__exit__ will call executor.shutdown(wait=True) which will block until all threads are done
  with futures.ThreadPoolExecutor(max_workers=3) as executor: # max_workers=3 to see pending futures in the output
    to_do: list[futures.Future] = []
    for cc in sorted(cc_list):
      future = executor.submit(download_one, cc)
      to_do.append(future)
      print(f"Scheduled for {cc}: {future}")

    for count, future in enumerate(futures.as_completed(to_do), 1):
      res: str = future.result()
      print(f"{future} result: {res!r}")

  return count

if __name__ == "__main__":
  main(download_many)



`concurrent.futures.as_completed` function takes an iterable of futures and returns an iterator that yields futures as they are done.

The higher-level `executor.map` is replaced by two `for` loops: one to create and schedule the futures, the other to retrieve their results. While we are at it, we'll add a few `print` calls to display each future before and after it's done.

In [7]:
!python flags_threadpool_futures.py

Scheduled for BR: <Future at 0x799001ff6260 state=running>
Scheduled for CN: <Future at 0x799001d70c10 state=running>
Scheduled for ID: <Future at 0x799001d71480 state=running>
Scheduled for IN: <Future at 0x799001d71d80 state=pending>
Scheduled for US: <Future at 0x799001d71db0 state=pending>
CN <Future at 0x799001d70c10 state=finished returned str> result: 'CN'
ID <Future at 0x799001d71480 state=finished returned str> result: 'ID'
BR <Future at 0x799001ff6260 state=finished returned str> result: 'BR'
IN <Future at 0x799001d71d80 state=finished returned str> result: 'IN'
US <Future at 0x799001d71db0 state=finished returned str> result: 'US'

5 downloads in 0.35s


Now, let's take a brief look at a simple way to work around the GIL for CPU-bound jobs using `concurrent.futures`.

## Launching Processes with concurrent.futures

The real value of `ProcessPoolExecutor` is in CPU-intensive jobs.

In [ ]:
# proc_pool.py

import sys
from concurrent import futures # hides multiprocessing / SimpleQueue / etc
from time import perf_counter
from typing import NamedTuple

from primes import is_prime, NUMBERS

class PrimeResult(NamedTuple):
  n: int
  flag: bool
  elapsed: float

def check(n: int) -> PrimeResult:
  t0 = perf_counter()
  res = is_prime(n)
  return PrimeResult(n, res, perf_counter() - t0)

def main() -> None:
  if len(sys.argv) < 2:
    workers = None
  else:
    workers = int(sys.argv[1])

  executor = futures.ProcessPoolExecutor(workers)
  actual_workers = executor._max_workers # type: ignore

  print(f"Checking {len(NUMBERS)} numbers with {actual_workers} processes:")

  t0 = perf_counter()

  numbers = sorted(NUMBERS, reverse=True)
  with executor:
    # executor.map() returns the result in the same order as the numbers are given
    for n, prime, elapsed in executor.map(check, numbers):
      label = 'P' if prime else ' '
      print(f"{n: 16}   {label} {elapsed:9.6f}s")

  time = perf_counter() - t0
  print(f"Total time: {time:.2f}s")

if __name__ == "__main__":
  main()

In [13]:
!python proc_pool.py

Checking 20 numbers with 2 processes:
 9999999999999999      0.000036s
 9999999999999917   P 15.200927s
 7777777777777777      0.000005s
 7777777777777753   P 13.473852s
 7777777536340681     13.214291s
 6666667141414921     12.325249s
 6666666666666719   P 12.355056s
 6666666666666666      0.000002s
 5555555555555555      0.000009s
 5555555555555503   P 11.451030s
 5555553133149889     12.853707s
 4444444488888889     11.632231s
 4444444444444444      0.000002s
 4444444444444423   P  9.667734s
 3333335652092209      8.738373s
 3333333333333333      0.000010s
 3333333333333301   P  6.512187s
 299593572317531   P  2.902751s
 142702110479723   P  1.590411s
               2   P  0.000002s
Total time: 66.87s
